## Maps Scraping for Baly

In [24]:
%%capture
pip install beautifulsoup4 requests pandas lxml

In [19]:
import requests
from bs4 import BeautifulSoup
import lxml.html
import re

You will need username and password, ask me for it.

In [25]:
username = input("username")
password = input("password")

username *****
password *****


This is where we search for a particular PoI - in this case restaurants in Najaf.

In [5]:
payload = {
    'source': 'google_maps',
    'query': 'restaurants near me',
    'user_agent_type': 'desktop',
    'domain': 'com',
    'geo_location': 'Najaf, Iraq',
    'start_page': '1',
    'pages': '3'
}
response = requests.request(
	'POST',
	'https://realtime.oxylabs.io/v1/queries',
	auth=(username, password),
	json=payload,
	timeout=180
)
print(response.status_code)

200


In [5]:
payload = {
    'source': 'google_maps',
    'query': 'restaurants near me',
    'user_agent_type': 'desktop',
    'domain': 'com',
    'geo_location': 'Najaf, Iraq',
    'start_page': '1',
    'pages': '3'
}
response = requests.request(
	'POST',
	'https://realtime.oxylabs.io/v1/queries',
	auth=(username, password),
	json=payload,
	timeout=180
)
print(response.status_code)

200


In [10]:
results = response.json()['results']
html_files = [result['content'] for result in results]

In [20]:
name_selector = '[role="heading"]'
rating_selector = 'span[aria-hidden="true"]'
rating_count_selector = '[class*="RDApEe"]'
details_selector = '.rllt__details div:nth-of-type(5)'
lat_selector = '[data-lat]'
lng_selector = '[data-lng]'
type_selector = '//div[@class="rllt__details"]/div[2]/text()'
address_selector = '.rllt__details div:nth-of-type(3)'

count_match = re.search(r"\((.+)\)", rating_count_el.text)
rating_count = count_match.group(1) if count_match else ""


In [21]:
data = []
for html in html_files:
    soup = BeautifulSoup(html, 'html.parser')
    lxml_obj = lxml.html.fromstring(str(soup))
    index = -1

    for listing in soup.select('[class="VkpGBb"]'):
        index += 1
        place = listing.parent
        name_el = place.select_one(name_selector)
        name = name_el.text.strip() if name_el else ''
        
        rating_el = place.select_one(rating_selector)
        rating = rating_el.text.strip() if rating_el else ''
        
        rating_count_el = place.select_one(rating_count_selector)
        rating_count = ''
        if rating_count_el:
            count_match = re.search(r'\((.+)\)', rating_count_el.text)
            rating_count = count_match.group(1) if count_match else ''
        
        
        details_el = place.select_one(details_selector)
        details = details_el.text.strip() if details_el else ''
        
        lat_el = soup.select_one(lat_selector)
        lat = lat_el.get('data-lat') if lat_el else ''
        
        lng_el = soup.select_one(lng_selector)
        lng = lng_el.get('data-lng') if lng_el else ''
        
        type_el = lxml_obj.xpath(type_selector)
        place_types = []
        for item in type_el:
            parts = item.strip().split('·')
            non_empty_parts = [part.strip() for part in parts if part.strip()]
            if non_empty_parts:
                place_types.append(non_empty_parts[-1])
        
        address_el = place.select_one(address_selector)
        address = address_el.text.strip() if address_el else ''
        
        place = {
            'name': name,
            'place_type': place_types[index],
            'address': address,
            'rating': rating,
            'rating_count': rating_count,
            'latitude': lat,
            'longitude': lng,
            'details': details,
        }
        data.append(place)

In [22]:
print(data[0:5])

[{'name': 'دواجن العوادي', 'place_type': 'Chicken shop', 'address': '286G+7J6', 'rating': '', 'rating_count': '', 'latitude': '32.0106646', 'longitude': '44.3265272', 'details': ''}, {'name': 'مطعم السبطين عليه السلام', 'place_type': 'Restaurant', 'address': '286G+7J6', 'rating': '3.5', 'rating_count': '4', 'latitude': '32.0106646', 'longitude': '44.3265272', 'details': ''}, {'name': 'The Gate Restaurant', 'place_type': 'Restaurant', 'address': '286G+FMW، شارع البانزين خانة', 'rating': '3.5', 'rating_count': '488', 'latitude': '32.0106646', 'longitude': '44.3265272', 'details': ''}, {'name': 'مطعم أم الربيعين', 'place_type': 'Restaurant', 'address': '286G+7J6', 'rating': '5.0', 'rating_count': '3', 'latitude': '32.0106646', 'longitude': '44.3265272', 'details': ''}, {'name': 'مطعم دجاج بغداد', 'place_type': 'Restaurant', 'address': '286G+7J6 ابو صخير، شارع المستشفى', 'rating': '', 'rating_count': '', 'latitude': '32.0106646', 'longitude': '44.3265272', 'details': ''}]
